In [2]:
from empiricaldist import Pmf

In [3]:
coin = Pmf()
coin['heads'] = 1/2
coin['tails'] = 1/2
coin

,probs
heads,0.5
tails,0.5


In [4]:
die = Pmf.from_seq([1, 2, 3, 4, 5, 6])
die

,probs
1,0.166667
2,0.166667
3,0.166667
4,0.166667
5,0.166667
6,0.166667


In [5]:
letters = Pmf.from_seq(list('Mississippi'))
letters

,probs
M,0.090909
i,0.363636
p,0.181818
s,0.363636


In [6]:
die([1, 4, 7])

array([0.16666667, 0.16666667, 0.        ])